<a href="https://colab.research.google.com/github/Tiabet/DACON_COURT/blob/main/baseline_kollama2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    To login, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Token: 
Add token as git credential? (Y/n) Y
Token is valid (permission: write).
Cannot authenticate through git-credential as no helper is defined on your machine.
You might have to re-authenticate when pushing to the Hugging Face Hub.
Run the following command in your terminal in case you want to set the 'store' creden

In [ ]:
!pip install -q  torch peft==0.4.0 bitsandbytes==0.40.2 transformers==4.31.0 trl==0.4.7 accelerate sentence_transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.9/72.9 kB 2.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.5/92.5 MB 19.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 33.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.4/77.4 kB 9.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 280.0/280.0 kB 34.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.5/149.5 kB 20.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 100.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 536.7/536.7 kB 56.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.8/132.8 kB 18.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.8/132.8 kB 19.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 13.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 16

In [ ]:
from tqdm import tqdm
import pandas as pd
from datasets import load_dataset, Dataset
import numpy as np
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    TrainingArguments,
    pipeline
)
from peft import LoraConfig
from trl import SFTTrainer
import torch

model_name = "beomi/kollama-7b"

In [ ]:
# CUDA 사용 가능 여부 확인
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

Using device: cuda


In [ ]:
data = pd.read_csv('drive/MyDrive/DACON_LLM/train.csv')

In [ ]:
# 토크나이저 로드
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"  # Fix for fp16

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/231 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.69M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/93.0 [00:00<?, ?B/s]

In [ ]:
# 데이터 포맷팅 및 토크나이징
formatted_data = []
for _, row in tqdm(data.iterrows()):
    for q_col in ['질문_1', '질문_2']:
        for a_col in ['답변_1', '답변_2', '답변_3', '답변_4', '답변_5']:
            input_text = tokenizer.bos_token+'[INST]'+row[q_col] +'[/INST]'  + row[a_col] +tokenizer.eos_token
            formatted_data.append(input_text)
print('Done.')
dataset = Dataset.from_dict({"text": formatted_data})

644it [00:00, 7059.55it/s]

Done.


In [ ]:
# Quantization Config
quant_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_use_double_quant=False
)

# Model
base_model = AutoModelForCausalLM.from_pretrained(
    model,
    quantization_config=quant_config,
    device_map={"": 0}
)
base_model.config.use_cache = False
base_model.config.pretraining_tp = 1

config.json:   0%|          | 0.00/507 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/26.8k [00:00<?, ?B/s]

model-00001-of-00015.safetensors:   0%|          | 0.00/965M [00:00<?, ?B/s]

model-00002-of-00015.safetensors:   0%|          | 0.00/990M [00:00<?, ?B/s]

model-00003-of-00015.safetensors:   0%|          | 0.00/967M [00:00<?, ?B/s]

...

model-00013-of-00015.safetensors:   0%|          | 0.00/967M [00:00<?, ?B/s]

model-00014-of-00015.safetensors:   0%|          | 0.00/742M [00:00<?, ?B/s]

model-00015-of-00015.safetensors:   0%|          | 0.00/426M [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/15 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/137 [00:00<?, ?B/s]

In [ ]:
# LoRA Config
peft_parameters = LoraConfig(
    lora_alpha=16,
    lora_dropout=0.1,
    r=32,
    bias="none",
    task_type="CAUSAL_LM"
)

# Training Params
train_params = TrainingArguments(
    output_dir="Tiabet/Tiabet-llama2-finetuned-epoch3",
    num_train_epochs=5,
    per_device_train_batch_size=4,
    gradient_accumulation_steps=1,
    optim="paged_adamw_32bit",
    save_steps=1600,
    logging_steps=500,
    learning_rate=2e-4,
    weight_decay=0.001,
    fp16=False,
    bf16=False,
    max_grad_norm=0.3,
    max_steps=-1,
    warmup_ratio=0.03,
    group_by_length=True,
    lr_scheduler_type="constant",
    report_to="tensorboard"
)

# Trainer
fine_tuning = SFTTrainer(
    model=base_model,
    train_dataset=dataset,
    peft_config=peft_parameters,
    dataset_text_field="text",
    tokenizer=tokenizer,
    args=train_params
)

# Training
fine_tuning.train()

# Save Model
fine_tuning.model.push_to_hub("Tiabet-kollama2-finetuned-epoch3")

/usr/local/lib/python3.10/dist-packages/peft/utils/other.py:102: FutureWarning: prepare_model_for_int8_training is deprecated and will be removed in a future version. Use prepare_model_for_kbit_training instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:159: UserWarning: You didn't pass a `max_seq_length` argument to the SFTTrainer, this will default to 1024
  warnings.warn(


Map:   0%|          | 0/6440 [00:00<?, ? examples/s]

You're using a PreTrainedTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(


Step,Training Loss
500,4.630300
1000,3.896300
1500,3.547000
2000,3.252400
2500,3.046100
3000,2.889300
3500,2.639800
4000,2.492200
4500,2.405700
5000,2.257800


/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:429: UserWarning: torch.

adapter_model.bin:   0%|          | 0.00/67.2M [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/Tiabet/Tiabet-kollama2-finetuned-epoch3/commit/0f1b781de4076dbde15a1fae1eb9fb35e5d584e3', commit_message='Upload model', commit_description='', oid='0f1b781de4076dbde15a1fae1eb9fb35e5d584e3', pr_url=None, pr_revision=None, pr_num=None)

inference

In [ ]:
from peft import PeftModel, PeftConfig
from transformers import AutoModelForCausalLM

config = PeftConfig.from_pretrained("Tiabet/Tiabet-kollama2-finetuned-epoch3")
model = AutoModelForCausalLM.from_pretrained("beomi/kollama-7b")
model = PeftModel.from_pretrained(model, "Tiabet/Tiabet-kollama2-finetuned-epoch3")

adapter_config.json:   0%|          | 0.00/438 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/507 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/26.8k [00:00<?, ?B/s]

model-00001-of-00015.safetensors:   0%|          | 0.00/965M [00:00<?, ?B/s]

model-00002-of-00015.safetensors:   0%|          | 0.00/990M [00:00<?, ?B/s]

...

model-00014-of-00015.safetensors:   0%|          | 0.00/742M [00:00<?, ?B/s]

model-00015-of-00015.safetensors:   0%|          | 0.00/426M [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/15 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/137 [00:00<?, ?B/s]

adapter_model.bin:   0%|          | 0.00/67.2M [00:00<?, ?B/s]

In [ ]:
import pandas as pd
from transformers import PreTrainedTokenizerFast
from tqdm import tqdm

tokenizer = PreTrainedTokenizerFast.from_pretrained("beomi/kollama-7b")

# Inference를 위한 test.csv 파일 로드
test = pd.read_csv('drive/MyDrive/DACON_LLM/test.csv')


In [ ]:
model.to(device)
# test.csv의 '질문'에 대한 '답변'을 저장할 리스트
preds = []
# '질문' 컬럼의 각 질문에 대해 답변 생성
for test_question in tqdm(test['질문']):
    # 입력 텍스트를 토큰화하고 모델 입력 형태로 변환
    input_ids = tokenizer.encode(tokenizer.bos_token + '[INST]' + test_question +'[/INST]', return_tensors='pt')

    # 답변 생성
    output_sequences = model.generate(
        input_ids=input_ids.to(device),
        max_length=300,
        temperature=0.9,
        top_k=1,
        top_p=0.8,
        num_beams=1,
        length_penalty=1.2,
        repetition_penalty=1.2,
        do_sample=True,
        num_return_sequences=1
    )

    # 생성된 텍스트(답변) 저장
    for generated_sequence in output_sequences:
        full_text = tokenizer.decode(generated_sequence, skip_special_tokens=False)
        # 질문과 답변의 사이를 나타내는 eos_token (</s>)를 찾아, 이후부터 출력
        answer_start = full_text.find('[/INST]') + len('[/INST]')
        answer_only = full_text[answer_start:].strip()
        answer_only = answer_only.replace('\n', ' ')
        preds.append(answer_only)

SyntaxError: invalid syntax. Perhaps you forgot a comma? (<ipython-input-20-07773fee03e5>, line 17)

In [ ]:
# Test 데이터셋의 모든 질의에 대한 답변으로부터 512 차원의 Embedding Vector 추출
# 평가를 위한 Embedding Vector 추출에 활용하는 모델은 'distiluse-base-multilingual-cased-v1' 이므로 반드시 확인해주세요.
from sentence_transformers import SentenceTransformer # SentenceTransformer Version 2.2.2

# Embedding Vector 추출에 활용할 모델(distiluse-base-multilingual-cased-v1) 불러오기
model = SentenceTransformer('distiluse-base-multilingual-cased-v1')

# 생성한 모든 응답(답변)으로부터 Embedding Vector 추출
pred_embeddings = model.encode(preds)
pred_embeddings.shape

.gitattributes:   0%|          | 0.00/690 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

2_Dense/config.json:   0%|          | 0.00/114 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/1.58M [00:00<?, ?B/s]

README.md:   0%|          | 0.00/2.47k [00:00<?, ?B/s]

...

tokenizer.json:   0%|          | 0.00/1.96M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/452 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/996k [00:00<?, ?B/s]

modules.json:   0%|          | 0.00/341 [00:00<?, ?B/s]

(130, 512)

In [ ]:
submit = pd.read_csv('drive/MyDrive/DACON_LLM/sample_submission.csv')
# 제출 양식 파일(sample_submission.csv)을 활용하여 Embedding Vector로 변환한 결과를 삽입
submit.iloc[:,1:] = pred_embeddings
submit.head()

,id,vec_0,vec_1,vec_2,vec_3,vec_4,vec_5,vec_6,vec_7,vec_8,...,vec_502,vec_503,vec_504,vec_505,vec_506,vec_507,vec_508,vec_509,vec_510,vec_511
0,TEST_000,-0.027148,0.058571,0.010376,0.005850,0.101920,-0.004139,0.013787,-0.009578,-0.021216,...,-0.023452,-0.032118,-0.003352,-0.049235,-0.039919,0.032349,0.014506,-0.029834,-0.019290,0.061307
1,TEST_001,-0.034024,-0.014175,-0.041230,0.011618,0.102943,0.007033,0.053830,-0.010049,-0.024299,...,-0.008694,-0.035625,0.067584,-0.004946,-0.024392,0.043755,0.040471,-0.021104,0.000820,0.015524
2,TEST_002,0.001269,-0.038478,-0.005728,-0.006738,0.142759,-0.001008,-0.006994,-0.012265,0.041841,...,-0.011662,-0.024751,0.062949,-0.016439,-0.013790,0.013332,-0.019483,0.020898,-0.038004,0.051246
3,TEST_003,0.022936,0.075898,0.010422,0.022045,0.065408,-0.038557,-0.056480,-0.064738,-0.022910,...,-0.040808,-0.002205,0.039815,-0.036658,-0.028517,0.036365,-0.013605,0.002583,-0.020453,0.002079
4,TEST_004,-0.017441,0.023326,0.003493,0.010740,0.111713,-0.021831,0.045830,-0.024057,-0.003542,...,-0.002355,-0.055153,0.050774,0.001023,-0.006733,0.053423,0.000876,-0.016567,-0.000591,0.041398


In [ ]:
# 리더보드 제출을 위한 csv파일 생성
submit.to_csv('drive/MyDrive/DACON_LLM/kollama2_baseline_submit.csv', index=False)

In [ ]:
preds[0]

'방청페인트는 주로 철재의 부식을 사용하여 광택을 최대한itable 가구를 선택하는 것으로 알려져 있습니다. 이러한 방청도료나 재료를 사용하거나 다양한 색상과 장식용 도구와 함께 다듬고 내구성을 높이는 것이 중요합니다. 그러나 방철벽돌, 콘크리트 등에 사용되는 발포탄글과 같은 종류가 있는 경우에도 고유한 재료로 활용됩니다. 따라서 각각의 목조 및 벽면에 적용하여 외부로 보호할 수 있도록 하는 것도 좋은 방법입니다. 이러한 특성과 다른 구조와 목적에 따라 적합한 기능이나 조명을 선택해야 합니다. 마지막으로, 접착제의 안정성, 화학물질 또는 기타 인장력을 고려하여 설계된 시공도 필요합니다. 또한, 난연, 규산, 그리고 자습한 표면의 강도를 유지하는데 도움이 됩니다. 이러한 방법을 통해 방독miterahy, 제단열을 이용하여 실내에서의 용도로 사용할 수도 있습니다. 이러한 종류의 방부제를 선택할 때에는 전문가의 도움을 받는 것이 좋습니다. 또한, 방염 시스템을 설치하기 위해 필요한 작업은 공간의 내부를 효율적으로 분리하고, 외부 환경에서도 매우 낮은 바닥에서 사용하고 있습니다. 이러한 방음재들을 조합하여 보다 많은 사람들에게 선호되는 공간을 완성시킬 수 있을 것입니다. 이러한 방수 기능을 제공하여 방NOL 공기를oren가지는 SUCH DAMAGE가 발생하지 않도록'

In [ ]:
test['질문'][0]

'방청 페인트의 종류에는 어떤 것들이 있는지 알고 계신가요? 또한, 원목사이딩을 사용하는 것에 어떤 단점이 있을까요?'